# Projet NexFloox
## Systeme de Recommandation


### Importation des librairies


In [1]:
# Importation des librairies

import pandas as pd
import numpy as np
import os
import re
import time
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine

### Importation des variables d'environnement

In [2]:
load_dotenv(r'variable_env/VAR.env')
nom_base_de_donnees = os.getenv("NOM_BASES_DONNEES")
utilisateur = os.getenv("USERAZURE")
mot_de_passe = os.getenv("PASSWORD")   
host = os.getenv("HOST")
port = os.getenv("PORT")
path_donnees = os.getenv("PATH_DONNEES")

### Connexion à la base de données (using sqlalchemy2)

In [3]:
# URL de connexion PostgreSQL
DATABASE_URL = f"postgresql://{utilisateur}:{mot_de_passe}@{host}:{port}/{nom_base_de_donnees}"
# Créer un moteur SQLAlchemy (engine)
engine = create_engine(DATABASE_URL)

# connexion à la BDD
try:
    with engine.connect() as connection:
        print("Connexion réussie à PostgreSQL avec SQLAlchemy !")
except Exception as e:
    print("Erreur de connexion :", e)


Connexion réussie à PostgreSQL avec SQLAlchemy !


### Extraction des features interessantes

In [4]:
# Ceci est l'exemple d'extraction sql vu avec Remi en cours

SQL = '''
WITH prep_ratings AS (

SELECT tb_ratings.tconst AS id_film
, "averageRating" AS target
, "numVotes"
, "titleType"
, "genres"
, split_part("genres", ',', 1) AS genre_1
, split_part("genres", ',', 2) AS genre_2
, split_part("genres", ',', 3) AS genre_3
, "isAdult"
, "startYear"
-- Données appartenant à la table title_crew
, split_part("directors", ',', 1) AS id_director_1
, split_part("directors", ',', 2) AS id_director_2
, split_part("directors", ',', 3) AS id_director_3
, split_part("writers", ',', 1) AS id_writer_1
-- Données appartenant à la table name_basics

FROM cyril_netfloox."title_ratings" AS tb_ratings
LEFT JOIN cyril_netfloox."title_basics" AS tb_basics ON tb_ratings.tconst = tb_basics.tconst
LEFT JOIN cyril_netfloox."title_crew" AS tb_crew ON tb_ratings.tconst = tb_crew.tconst
-- On veut faire correspondre les id_directors à la tables des noms de ces personnes, on va utiliser les CTE 
-- en debut de requete avec WITH on encapsule notre code prededant dans une table temporaire.
--LEFT JOIN cyril_netfloox."name_basics" AS tb_name ON tb_crew.tconst = tb_name.nconst

)

SELECT prep_ratings.*
, tb_name_1."primaryName" AS director_1
, tb_name_2."primaryName" AS director_2
, tb_name_3."primaryName" AS director_3
, tb_name_4."primaryName" AS writer_1
FROM prep_ratings
LEFT JOIN cyril_netfloox."name_basics" AS tb_name_1 ON prep_ratings.id_director_1 = tb_name_1.nconst
LEFT JOIN cyril_netfloox."name_basics" AS tb_name_2 ON prep_ratings.id_director_2 = tb_name_2.nconst
LEFT JOIN cyril_netfloox."name_basics" AS tb_name_3 ON prep_ratings.id_director_3 = tb_name_3.nconst
LEFT JOIN cyril_netfloox."name_basics" AS tb_name_4 ON prep_ratings.id_writer_1 = tb_name_4.nconst;
'''

#df = pd.read_sql(SQL, engine)
#print(df)


In [5]:
# !!!ATTENTION!!! ce test sera à réitérer sur la table entière
# Mais il y a peu de chances qu'il y ait d'autres roles interessants

SQL = '''
SET search_path TO echantillon_test; -- A renouveller à chaque requête SQL
SELECT DISTINCT category
FROM title_principals
;
'''

df = pd.read_sql(SQL, engine)
connection.close()
print("on regarde les differentes catégories existantes pour ensuite créer les colonnes")
print(df)

on regarde les differentes catégories existantes pour ensuite créer les colonnes
               category
0                  self
1                writer
2                editor
3              composer
4      casting_director
5   production_designer
6       cinematographer
7              director
8       archive_footage
9               actress
10             producer
11                actor


In [6]:

SQL = '''
SET search_path TO echantillon_test; -- A renouveller à chaque requête SQL
SELECT DISTINCT region
FROM title_akas
;
'''

df = pd.read_sql(SQL, engine)
connection.close()

print("on regarde les differentes régions existantes pour ensuite créer les colonnes")
print(df)

on regarde les differentes régions existantes pour ensuite créer les colonnes
   region
0      BE
1      CO
2      DZ
3      AZ
4      RU
..    ...
86     TR
87     CU
88     AL
89     HR
90     GR

[91 rows x 1 columns]


In [7]:
SQL='''
SET search_path TO echantillon_test; -- A renouveller à chaque requête SQL
SELECT "titleId"
, STRING_AGG(region, ', ') AS region
FROM title_akas
GROUP BY "titleId"
'''
df = pd.read_sql(SQL, engine)
connection.close()

print(df.head(20))

      titleId                                             region
0   tt0000001                         DE, US, HU, GR, RU, UA, JP
1   tt0000002                         HU, RO, DE, FR, US, RU, JP
2   tt0000003                 GB, US, DE, FR, RO, HU, RU, UA, JP
3   tt0000004                             HU, DE, FR, RO, RU, JP
4   tt0000005         CA, US, HU, US, GB, US, US, DE, GB, UA, RU
5   tt0000006                                 US, DE, PT, HU, RU
6   tt0000007                 HU, US, DE, MX, US, US, RU, UA, JP
7   tt0000008                 HU, US, US, DE, DE, US, RU, UA, JP
8   tt0000009                                     AU, HU, US, DE
9   tt0000010  AR, MX, CA, FR, BR, DE, US, US, IT, ES, FR, FR...
10  tt0000011                                             DE, RU
11  tt0000012  GB, US, XEU, XWW, XWW, CA, FR, BR, HU, DE, TR,...
12  tt0000013  GB, US, XWW, HU, FR, ES, RS, FR, FR, FR, PL, U...
13  tt0000014  HU, FR, DE, ES, DK, FR, IT, FR, FR, BR, PL, CZ...
14  tt0000015            

In [8]:
pd.set_option("display.max_columns", None)

In [9]:
SQL = '''
SET search_path TO echantillon_test; -- A renouveller à chaque requête SQL

WITH tb_roles AS (
--SET search_path TO echantillon_test; -- A renouveller à chaque requête SQL
SELECT tconst
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'self') AS self
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'writer') AS writer
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'editor') AS editor
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'composer') AS composer
--, STRING_AGG(nconst, ',') FILTER (WHERE category = 'casting_director') AS casting_director
--, STRING_AGG(nconst, ',') FILTER (WHERE category = 'production_designer') AS production_designer
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'cinematographer') AS cinematographer
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'director') AS director
--, STRING_AGG(nconst, ',') FILTER (WHERE category = 'archive_footage') AS archive_footage
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'actress') AS actress
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'producer') AS producer
, STRING_AGG(nconst, ',') FILTER (WHERE category = 'actor') AS actor
FROM title_principals
GROUP BY tconst
ORDER BY tconst
),

tb_regions AS (
SELECT "titleId"
, STRING_AGG(region, ',') AS region
FROM title_akas
GROUP BY "titleId"
)

SELECT tb_film.tconst
, "primaryTitle"
, "titleType"
, "isAdult"
, "startYear"
, "genres"

-- Dats from title_ratings
, "averageRating"
, "numVotes"

-- Datas from  title_crew
--, split_part("directors", ',', 1) AS id_director_1
--, split_part("directors", ',', 2) AS id_director_2
--, split_part("directors", ',', 3) AS id_director_3
--, split_part("writers", ',', 1) AS id_writer_1
--, split_part("writers", ',', 2) AS id_writer_2
--, split_part("writers", ',', 3) AS id_writer_3
-- Datas from tb_langages

-- Datas from tb_regions
, region

-- Datas from tb_roles
, self
, writer
, editor
, composer
, cinematographer
, director
, actress
, producer
, actor

FROM title_basics AS tb_film
LEFT JOIN title_ratings AS tb_notes ON tb_notes.tconst = tb_film.tconst
LEFT JOIN title_crew AS tb_crew ON tb_crew.tconst = tb_film.tconst
LEFT JOIN tb_regions ON tb_regions."titleId" = tb_film.tconst
LEFT JOIN tb_roles ON tb_film.tconst = tb_roles.tconst
;
'''
df = pd.read_sql(SQL, engine)
connection.close()


print(df.shape)
print(df.head())



(100000, 18)
      tconst            primaryTitle titleType  isAdult  startYear  \
0  tt0000001              Carmencita     short        0     1894.0   
1  tt0000002  Le clown et ses chiens     short        0     1892.0   
2  tt0000003            Poor Pierrot     short        0     1892.0   
3  tt0000004             Un bon bock     short        0     1892.0   
4  tt0000005        Blacksmith Scene     short        0     1893.0   

                     genres  averageRating  numVotes  \
0         Documentary,Short            5.7    2121.0   
1           Animation,Short            5.6     286.0   
2  Animation,Comedy,Romance            6.4    2160.0   
3           Animation,Short            5.3     183.0   
4                     Short            6.2    2888.0   

                             region       self writer     editor   composer  \
0              DE,US,HU,GR,RU,UA,JP  nm1588970   None       None       None   
1              HU,RO,DE,FR,US,RU,JP       None   None       None  nm133

##### On va nettoyer nos colonnes en remplacant les Nan par du texte vide, on met en minucuscule, puis on splitte en liste les colonnes contenant du texte séparé par des virgules


In [10]:
# On stocke dans df_clean

df_clean = df
colonnes_string_to_list = ['genres','region','self','writer','editor','composer','cinematographer','director','actress','producer','actor']

for col in colonnes_string_to_list:
    
    df_clean[col] = df[col].fillna("")
    df_clean[col] = df_clean[col].str.lower()
    df_clean[col] = df_clean[col].apply(lambda x: x.split(","))

df[colonnes_string_to_list].head(20)
df_clean.head()

,tconst,primaryTitle,titleType,isAdult,startYear,genres,averageRating,numVotes,region,self,writer,editor,composer,cinematographer,director,actress,producer,actor
0,tt0000001,Carmencita,short,0,1894.0,"[documentary, short]",5.7,2121.0,"[de, us, hu, gr, ru, ua, jp]",[nm1588970],[],[],[],[nm0374658],[nm0005690],[],[nm0005690],[]
1,tt0000002,Le clown et ses chiens,short,0,1892.0,"[animation, short]",5.6,286.0,"[hu, ro, de, fr, us, ru, jp]",[],[],[],[nm1335271],[],[nm0721526],[],[],[]
2,tt0000003,Poor Pierrot,short,0,1892.0,"[animation, comedy, romance]",6.4,2160.0,"[gb, us, de, fr, ro, hu, ru, ua, jp]",[],[],[nm5442200],[nm1335271],[],[nm0721526],[],"[nm1770680, nm0721526]",[]
3,tt0000004,Un bon bock,short,0,1892.0,"[animation, short]",5.3,183.0,"[hu, de, fr, ro, ru, jp]",[],[],[],[nm1335271],[],[nm0721526],[],[],[]
4,tt0000005,Blacksmith Scene,short,0,1893.0,[short],6.2,2888.0,"[ca, us, hu, us, gb, us, us, de, gb, ua, ru]",[],[],[],[],[],[],[],[nm0249379],"[nm0443482, nm0653042]"


In [11]:
# On extraire les noms de personnes des listes et les mettre dans des colonnes 
# par role director_1, director_2, ... 

# # On stocke dans df_final

colonnes_list_to_explode = ['self','writer','editor','composer','cinematographer','director','actress','producer','actor']
df_final = df_clean[['tconst','primaryTitle','titleType','isAdult','startYear','genres','averageRating','numVotes','region']]
for col in colonnes_list_to_explode:
    
    # Transformation avec explode() pour avoir une ligne par element de chaque liste de la colonne col
    df_exploded = df_clean.explode(col)
    # Ajout d'un index pour numéroter les colonnes à créer 
    df_exploded['index'] = df_exploded.groupby('tconst').cumcount() + 1
    df_exploded['index'] = df_exploded['index'].fillna(-1)
    df_exploded['index'] = df_exploded['index'].astype(int)
    # Pivot pour transformer les lignes en colonnes
    df_pivot = df_exploded.pivot_table(index="tconst", columns="index", values=col, aggfunc="first")
    # Renommer les colonnes
    df_pivot.columns = [f"{col}_{i}" for i in df_pivot.columns]
    # Réinitialiser l'index 
    df_pivot.reset_index(inplace=True)
    #On supprime la col 'tconst' pour ne pas qu'elle soit dupliquer lors de la concaténation
    df_pivot = df_pivot.drop('tconst', axis=1)
    #On remplace les cellules Nan par du texte vide
    df_pivot = df_pivot.fillna('')
    #On concatène dans un nouveau df_final
    df_final = pd.concat([df_final, df_pivot], axis=1)
df_final.head(10)

,tconst,primaryTitle,titleType,isAdult,startYear,genres,averageRating,numVotes,region,self_1,self_2,self_3,self_4,self_5,self_6,self_7,self_8,self_9,self_10,writer_1,writer_2,writer_3,writer_4,editor_1,editor_2,editor_3,editor_4,composer_1,composer_2,composer_3,composer_4,cinematographer_1,cinematographer_2,cinematographer_3,cinematographer_4,director_1,director_2,director_3,director_4,actress_1,actress_2,actress_3,actress_4,actress_5,actress_6,actress_7,actress_8,actress_9,actress_10,producer_1,producer_2,producer_3,producer_4,actor_1,actor_2,actor_3,actor_4,actor_5,actor_6,actor_7,actor_8,actor_9,actor_10,actor_11,actor_12,actor_13,actor_14
0,tt0000001,Carmencita,short,0,1894.0,"[documentary, short]",5.7,2121.0,"[de, us, hu, gr, ru, ua, jp]",nm1588970,,,,,,,,,,,,,,,,,,,,,,nm0374658,,,,nm0005690,,,,,,,,,,,,,,nm0005690,,,,,,,,,,,,,,,,,
1,tt0000002,Le clown et ses chiens,short,0,1892.0,"[animation, short]",5.6,286.0,"[hu, ro, de, fr, us, ru, jp]",,,,,,,,,,,,,,,,,,,nm1335271,,,,,,,,nm0721526,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,tt0000003,Poor Pierrot,short,0,1892.0,"[animation, comedy, romance]",6.4,2160.0,"[gb, us, de, fr, ro, hu, ru, ua, jp]",,,,,,,,,,,,,,,nm5442200,,,,nm1335271,,,,,,,,nm0721526,,,,,,,,,,,,,,nm1770680,nm0721526,,,,,,,,,,,,,,,,
3,tt0000004,Un bon bock,short,0,1892.0,"[animation, short]",5.3,183.0,"[hu, de, fr, ro, ru, jp]",,,,,,,,,,,,,,,,,,,nm1335271,,,,,,,,nm0721526,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,tt0000005,Blacksmith Scene,short,0,1893.0,[short],6.2,2888.0,"[ca, us, hu, us, gb, us, us, de, gb, ua, ru]",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,nm0249379,,,,nm0443482,nm0653042,,,,,,,,,,,,
5,tt0000006,Chinese Opium Den,short,0,1894.0,[short],5.0,204.0,"[us, de, pt, hu, ru]",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,tt0000007,Corbett and Courtney Before the Kinetograph,short,0,1894.0,"[short, sport]",5.3,898.0,"[hu, us, de, mx, us, us, ru, ua, jp]",,,,,,,,,,,,,,,,,,,,,,,nm0374658,,,,nm0005690,nm0374658,,,,,,,,,,,,,nm0005690,nm0249379,,,nm0179163,nm0183947,,,,,,,,,,,,
7,tt0000008,Edison Kinetoscopic Record of a Sneeze,short,0,1894.0,"[documentary, short]",5.4,2270.0,"[hu, us, us, de, de, us, ru, ua, jp]",,,,,,,,,,,,,,,,,,,,,,,nm0374658,,,,nm0005690,,,,,,,,,,,,,,nm0005690,,,,nm0653028,,,,,,,,,,,,,
8,tt0000009,Miss Jerry,movie,0,1894.0,[romance],5.4,218.0,"[au, hu, us, de]",,,,,,,,,,,nm0085156,,,,,,,,,,,,nm0085156,,,,nm0085156,,,,nm0063086,,,,,,,,,,nm0085156,,,,nm0183823,nm1309758,,,,,,,,,,,,
9,tt0000010,Leaving the Factory,short,0,1895.0,"[documentary, short]",6.8,7843.0,"[ar, mx, ca, fr, br, de, us, us, it, es, fr, f...",,,,,,,,,,,,,,,,,,,,,,,nm0525910,,,,nm0525910,,,,,,,,,,,,,,nm0525910,,,,,,,,,,,,,,,,,


#### REMARQUE : Ici on a créé des colonnes roles contenant un id_name mais peut-être il peut être intéressant de créer des colonnes par couple nom-role. Cela crée beaucoup plus de colonnes mais je pense que l'on perd moins d'informations sur les personnes. Dans le cas actuel si un id_name apparait en tant que director et actor, le modèle ne 'voit pas' que c'est la même personne. 

# Preprocessing


#### REMARQUE  Après plusieurs essais, il semble compliqué de travailler sur 'df_final' avec des colonnes ainsi formatées.. Je vais revenir sur mon 'df_clean' et appliquer un mlb()

In [12]:
df_clean.head()

,tconst,primaryTitle,titleType,isAdult,startYear,genres,averageRating,numVotes,region,self,writer,editor,composer,cinematographer,director,actress,producer,actor
0,tt0000001,Carmencita,short,0,1894.0,"[documentary, short]",5.7,2121.0,"[de, us, hu, gr, ru, ua, jp]",[nm1588970],[],[],[],[nm0374658],[nm0005690],[],[nm0005690],[]
1,tt0000002,Le clown et ses chiens,short,0,1892.0,"[animation, short]",5.6,286.0,"[hu, ro, de, fr, us, ru, jp]",[],[],[],[nm1335271],[],[nm0721526],[],[],[]
2,tt0000003,Poor Pierrot,short,0,1892.0,"[animation, comedy, romance]",6.4,2160.0,"[gb, us, de, fr, ro, hu, ru, ua, jp]",[],[],[nm5442200],[nm1335271],[],[nm0721526],[],"[nm1770680, nm0721526]",[]
3,tt0000004,Un bon bock,short,0,1892.0,"[animation, short]",5.3,183.0,"[hu, de, fr, ro, ru, jp]",[],[],[],[nm1335271],[],[nm0721526],[],[],[]
4,tt0000005,Blacksmith Scene,short,0,1893.0,[short],6.2,2888.0,"[ca, us, hu, us, gb, us, us, de, gb, ua, ru]",[],[],[],[],[],[],[],[nm0249379],"[nm0443482, nm0653042]"


In [13]:
col_target = 'tconst'
col_titre_text = 'primaryTitle'
col_num = ['isAdult', 'startYear', 'averageRating', 'numVotes']
col_title_type = 'titleType'
col_list_text = ['genres', 'region', 'self', 'writer', 'editor','composer', 'cinematographer', 'director', 'actress', 'producer','actor']
#col_list_text = 'self'

Le code ci-dessous est inutile.

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

mlb = MultiLabelBinarizer()
countv = CountVectorizer()
le = LabelEncoder()
tfidfv = TfidfVectorizer()
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)

#def myMLB(X):
#    mlb = MultiLabelBinarizer()
#    return mlb.fit_transform(X)
#text_list_transformer = FunctionTransformer(myMLB, validate=False)


text_list_transformer = Pipeline(steps = [
    ('mlb', MultiLabelBinarizer())
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_list_transformer, col_list_text)
    ])


data_final1 = pd.DataFrame()
data_final1['label_encoded'] = le.fit_transform(df_clean['tconst'])


# On applique un mlb a toutes les colonnes texte excepte le titre
for col in col_list_text:
    transformed_data = mlb.fit_transform(df_clean[col])
    #reduced_data = svd.fit_transform(transformed_data)
    data_final1 = pd.concat([data_final1, pd.DataFrame(transformed_data)], axis=1)
    print(data_final1.shape)
    
#data_final1.columns = data_final1.columns.astype(str)
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
pca = PCA(n_components=2)

#reduced_data_final = svd.fit_transform(data_final1)

#print(data_final1.shape)   
#print(data_final1.head())
#print("--------")
print("reduced")
#print(reduced_data_final.shape)
#print(reduced_data_final)

# On travaille sur la colonne titre 'primaryTitle'
# On essaie un countVectorizer() mais on pourra aussi essayer un tfidfVectorizer()
#transformed_data = countv.fit_transform(df_clean[col_titre_text])
#df_transformed_data = pd.DataFrame(transformed_data.toarray(), columns = countv.get_feature_names_out())
#data_final = pd.concat([data_final, df_transformed_data], axis=1)
#print(data_final.shape)
#print(df_transformed_data.head())

#transformed_data = le.fit_transform(df_clean[col_title_type])
#df_transformed_data = pd.DataFrame(transformed_data)
#data_final1 = pd.concat([data_final1, df_transformed_data], axis=1)
#print(df_transformed_data)
#stdscl = StandardScaler()
#transformed_data = stdscl.fit_transform(df_clean[col_num])
#data_final1 = pd.concat([data_final1, pd.DataFrame(transformed_data)], axis=1)
#print(data_final.shape)



(100000, 30)
(100000, 121)
(100000, 362)
(100000, 3581)
(100000, 3732)
(100000, 3912)
(100000, 4701)
(100000, 5995)
(100000, 11089)
(100000, 11672)
(100000, 19404)
reduced


## La cellule suivante est commentée. Elle fait planter le noyau car la matrice résultante fait 100000 x 100000 ! Ayant fait mes premiers essais sur un dataset de 10000 lignes, ça passait mais là ... Ca coince !!!

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

'''
# Création de la matrice de similarité
cosine_sim = cosine_similarity(data_final1.fillna(0), data_final1.fillna(0))

# Création d'un DataFrame pour la lisibilité
df_similarity = pd.DataFrame(cosine_sim, index=df_clean["primaryTitle"], columns=df_clean["primaryTitle"])

# Affichage de la matrice de similarité
print(df_similarity)
df_similarity['Carmencita'].sort_values(ascending=False)
# Récupérer les classes des labels
mlb = MultiLabelBinarizer()
mlb.fit(df_clean[col_list_text])
column_names = mlb.classes_

# Convertir la sortie transformée en DataFrame pour affichage
transformed_df = pd.DataFrame(transformed_data, columns=column_names)

# Affichage du DataFrame transformé
print(transformed_df)
'''

'\n# Création de la matrice de similarité\ncosine_sim = cosine_similarity(data_final1.fillna(0), data_final1.fillna(0))\n\n# Création d\'un DataFrame pour la lisibilité\ndf_similarity = pd.DataFrame(cosine_sim, index=df_clean["primaryTitle"], columns=df_clean["primaryTitle"])\n\n# Affichage de la matrice de similarité\nprint(df_similarity)\ndf_similarity[\'Carmencita\'].sort_values(ascending=False)\n# Récupérer les classes des labels\nmlb = MultiLabelBinarizer()\nmlb.fit(df_clean[col_list_text])\ncolumn_names = mlb.classes_\n\n# Convertir la sortie transformée en DataFrame pour affichage\ntransformed_df = pd.DataFrame(transformed_data, columns=column_names)\n\n# Affichage du DataFrame transformé\nprint(transformed_df)\n'

## Même problème d'où la cellule suivante qui est commentée !!!

In [16]:
#
# On essaie une autre approche  consistant a concatener en une seule colonne 
#les colonnes contenant du texte, chaque mot separe par un espace

'''
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import Normalizer, StandardScaler
import numpy as np

data = df[['tconst', 'primaryTitle', 'genres', 'averageRating', 'numVotes', 'startYear', 'isAdult', 'titleType']]

# Créer une nouvelle colonne 'features' qui combine les informations textuelles
#col_features = ['director', 'producer', 'actor', 'actress', 'primaryTitle', 'genres']
col_features = ['director', 'producer', 'actor', 'actress', 'genres']
data.loc[:,'features'] = ""
for col in col_features:
    data.loc[:, col] = df[col]
    data.loc[:, col] = data[col].fillna("")
    data.loc[:, 'features'] = data['features'] + "," + data[col].apply(lambda x: ' '.join(x if isinstance(x, list) else [str(x)])).fillna('')

data.loc[:, 'features'] = data['features'].apply(lambda x: x.replace(",", " ",))
data.loc[:, 'features'] = data['features'].str.strip()

# Ajouter des caractéristiques pour les rôles multiples
#data['multi_role'] = data.apply(lambda row: ' '.join([role for role in [row['actor'], row['director'], row['producer']] if row['actor'] in role]), axis=1)

# A voir si on traite d'autres stopwords...
# Utiliser TF-IDF pour vectoriser les caractéristiques textuelles
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['features'])
#print(tfidf_matrix)

##  La normalisation ne sert pas ici
# Normaliser les caractéristiques textuelles
#normalizer = Normalizer()
#tfidf_matrix_normalized = normalizer.fit_transform(tfidf_matrix)
#print(tfidf_matrix_normalized)

col_numeric = ['averageRating', 'numVotes']
data[col_numeric] = data[col_numeric].fillna(-1)
scaler = StandardScaler()
numeric_features = scaler.fit_transform(data[col_numeric])

#numeric_features_normalized = normalizer.fit_transform(numeric_features)
#print(numeric_features_normalized)
# Combiner les caractéristiques textuelles et numériques
#combined_features = np.hstack((tfidf_matrix_normalized.toarray(), numeric_features))

# Calculer la similarité cosinus entre les films
#cosine_sim = cosine_similarity(combined_features, combined_features)
cosine_sim = cosine_similarity(tfidf_matrix.toarray(), tfidf_matrix.toarray())

print(cosine_sim)
# Fonction pour obtenir les recommandations
'''

'\nimport pandas as pd\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.metrics.pairwise import cosine_similarity\nfrom sklearn.preprocessing import Normalizer, StandardScaler\nimport numpy as np\n\ndata = df[[\'tconst\', \'primaryTitle\', \'genres\', \'averageRating\', \'numVotes\', \'startYear\', \'isAdult\', \'titleType\']]\n\n# Créer une nouvelle colonne \'features\' qui combine les informations textuelles\n#col_features = [\'director\', \'producer\', \'actor\', \'actress\', \'primaryTitle\', \'genres\']\ncol_features = [\'director\', \'producer\', \'actor\', \'actress\', \'genres\']\ndata.loc[:,\'features\'] = ""\nfor col in col_features:\n    data.loc[:, col] = df[col]\n    data.loc[:, col] = data[col].fillna("")\n    data.loc[:, \'features\'] = data[\'features\'] + "," + data[col].apply(lambda x: \' \'.join(x if isinstance(x, list) else [str(x)])).fillna(\'\')\n\ndata.loc[:, \'features\'] = data[\'features\'].apply(lambda x: x.replace(",", " ",))\ndata.

## On revient a la premiere transfo
... Mais j'ai toujorus le même probleme de Kernel qui plante


In [17]:
'''
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

mlb = MultiLabelBinarizer()
countv = CountVectorizer()
le = LabelEncoder()
tfidfv = TfidfVectorizer()
#def myMLB(X):
#    mlb = MultiLabelBinarizer()
#    return mlb.fit_transform(X)
#text_list_transformer = FunctionTransformer(myMLB, validate=False)


text_list_transformer = Pipeline(steps = [
    ('mlb', MultiLabelBinarizer())
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_list_transformer, col_list_text)
    ])


data_final = pd.DataFrame()
print(data_final.shape)
data_final['label_encoded'] = le.fit_transform(df_clean['tconst'])
print(data_final.shape)

# On applique un mlb a toutes les colonnes texte excepte le titre
for col in col_list_text:
    transformed_data = mlb.fit_transform(df_clean[col])
    data_final = pd.concat([data_final, pd.DataFrame(transformed_data)], axis=1)

data_count = countv.fit_transform(data['features'])

data_final.columns = data_final.columns.astype(str)
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
#svd.fit(data_final)
pca = PCA(n_components=2)
#pca.fit(data_final)
#reduced_data_final = svd.fit_transform(data_final)
print("reduced")
print(reduced_data_final.shape)
print(reduced_data_final)
print("--------")
print(data_final.shape)   
print(data_final.head())


# On travaille sur la colonne titre 'primaryTitle'
# On essaie un countVectorizer() mais on pourra aussi essayer un tfidfVectorizer()
#transformed_data = countv.fit_transform(df_clean[col_titre_text])
#df_transformed_data = pd.DataFrame(transformed_data.toarray(), columns = countv.get_feature_names_out())
#data_final = pd.concat([data_final, df_transformed_data], axis=1)
#print(data_final.shape)
#print(df_transformed_data.head())

transformed_data = le.fit_transform(df_clean[col_title_type])
df_transformed_data = pd.DataFrame(transformed_data)
data_final = pd.concat([data_final, df_transformed_data], axis=1)
print(df_transformed_data)
stdscl = StandardScaler()
transformed_data = stdscl.fit_transform(df_clean[col_num])
data_final = pd.concat([data_final, pd.DataFrame(transformed_data)], axis=1)
print(data_final.shape)

cosine_sim = cosine_similarity(data_count, data_count)

# 4. Création d'un DataFrame pour la lisibilité
df_similarity = pd.DataFrame(cosine_sim, index=df_clean["tconst"], columns=df_clean["tconst"])

# Affichage de la matrice de similarité
print(df_similarity)
df_similarity['tt0000001'].sort_values(ascending=False)
'''

'\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.preprocessing import FunctionTransformer\nfrom sklearn.preprocessing import MultiLabelBinarizer\nfrom sklearn.feature_extraction.text import CountVectorizer\nfrom sklearn.preprocessing import StandardScaler, LabelEncoder\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.decomposition import TruncatedSVD\nfrom sklearn.decomposition import PCA\n\nmlb = MultiLabelBinarizer()\ncountv = CountVectorizer()\nle = LabelEncoder()\ntfidfv = TfidfVectorizer()\n#def myMLB(X):\n#    mlb = MultiLabelBinarizer()\n#    return mlb.fit_transform(X)\n#text_list_transformer = FunctionTransformer(myMLB, validate=False)\n\n\ntext_list_transformer = Pipeline(steps = [\n    (\'mlb\', MultiLabelBinarizer())\n])\n\n# Combine transformers\npreprocessor = ColumnTransformer(\n    transformers=[\n        (\'text\', text_list_transformer, col_list_text)\n    ])\n\n\ndata_final = pd.DataFrame(

# Solution qui fonctionne
Voici ci dessous une solution qui fonctionne. Mon erreur était de calculer une matrice de similarité complète. J'ai essayé de réduire en utilisant des fonction qui ne garde que la partie supérieure ou inférieure d'une matrice symétrique mais rien n'y faisait. 
Un monitoring système montrait que c'était l'espace mémoire qui saturait.De nombreux essais ont été effectué pour libérer de la mémoire en effaçant les DataFrames inutiles, en forçant le garbage collector, ... 

La solution m'a été finalement été suggéré par Jonathan (Merci) la veille la soutenance et consiste non pas à passer en paramètre 2 fois la matrice comme je le faisais mais la matrice et le vecteur du film dont on désire chercher la similarité. (Elémentaire mon cher Watson)


In [18]:
import pandas as pd
import numpy as np
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import fct_extract as my_fct
import gc

#df = my_fct.extraire_donnees_BDD()

#data = df[['tconst', 'primaryTitle', 'genres', 'averageRating', 'numVotes', 'startYear', 'isAdult', 'titleType']]
data = df[['primaryTitle', 'genres', 'averageRating', 'isAdult', 'titleType']]

# Créer une nouvelle colonne 'features' qui combine les informations textuelles
#col_features = ['director', 'producer', 'actor', 'actress', 'primaryTitle', 'genres']
col_features = ['director', 'producer', 'actor', 'actress', 'genres']
data.loc[:,'features'] = ""
for col in col_features:
    data.loc[:, col] = df[col]
    data.loc[:, col] = data[col].fillna("")
    data.loc[:, 'features'] = data['features'] + "," + data[col].apply(lambda x: ' '.join(x if isinstance(x, list) else [str(x)])).fillna('')
    # Je rajoute l'effacement de la colonne traitée
    data.drop(columns=col, inplace=True)
data.loc[:, 'features'] = data['features'].apply(lambda x: x.replace(",", " ",))
data.loc[:, 'features'] = data['features'].str.strip()

def get_index_from_title(df, title):
  # simple exemple a adapter
  #print(df[df['primaryTitle'].fillna('').str.contains(title)].index)
 
  return df[df['primaryTitle'].fillna('').str.contains(title)]

vectorizer = CountVectorizer()
svd = TruncatedSVD(n_components=10)  # Ajustez le nombre de composants selon vos besoins
pipeline = make_pipeline(vectorizer, svd)

# Transformation des caractéristiques textuelles
features_transformed = pipeline.fit_transform(data['features'])
# On passe de float64 a float16
features_transformed = features_transformed.astype(np.float16)

# On sauvegarde la matrice de transformation
file = 'features_transformed.npy'
np.save(file, features_transformed)

# On choisit un titre
title = 'Enchanted Cup'
movie_idx = get_index_from_title(df, title).index[0] # Index[0] permet de choisir le premier film qui sort

X = np.load(file)
Y = features_transformed[movie_idx, :]

# calcule des similarités
cosine_sim = cosine_similarity(X, Y.reshape(1,-1))

# On récupère le n films les plus semblables
# Pas encore tout compris mais ca marche ! ;-) 
n=5
cosine_sim_1d = cosine_sim.flatten()
idxs = np.argpartition(cosine_sim_1d, -n)[-n:]
idxs = idxs[np.argsort(cosine_sim_1d[idxs])][::-1]

# affichage des recommandations
print('Recommandations : ')
print(df.loc[idxs])

/var/folders/lm/5jb7d18d2_gfyv8y4w2tsg4r0000gn/T/ipykernel_13493/904268893.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,'features'] = ""
/var/folders/lm/5jb7d18d2_gfyv8y4w2tsg4r0000gn/T/ipykernel_13493/904268893.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, col] = df[col]
/var/folders/lm/5jb7d18d2_gfyv8y4w2tsg4r0000gn/T/ipykernel_13493/904268893.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docu

Recommandations : 
        tconst                    primaryTitle titleType  isAdult  startYear  \
388  tt0000392               The Enchanted Cup     short        0     1903.0   
216  tt0000219  Aladdin and the Wonderful Lamp     short        0     1899.0   
193  tt0000196        Faust and Mephistopheles     short        0     1898.0   
430  tt0000434                 Dorothy's Dream     short        0     1903.0   
178  tt0000181                      Cinderella     short        0     1898.0   

               genres  averageRating  numVotes        region self  \
388  [fantasy, short]            NaN       NaN          [gb]   []   
216  [fantasy, short]            6.6      32.0          [gb]   []   
193  [fantasy, short]            4.8      32.0          [gb]   []   
430  [fantasy, short]            NaN       NaN      [ru, gb]   []   
178  [fantasy, short]            6.6      62.0  [ve, gb, ru]   []   

          writer editor composer cinematographer     director      actress  \
388    

# Conclusion :

Et bien c'est pas terrible ! Tout d'abord il est difficle de juger des films similaires proposé car on ne connait pas les films. Cependant, ici, en comparant les features de notre film avec les autres on voit que les seuls similarités sont surtout au niveau du genre. Je n'ai pas eu el temsp d'ajoueter d'autres features comme je l'avais fait orinelelement avec un dataset plus petit car j'ai ensuite tenté de récupérer des données plus récentes de la database complète qui contient 18 Millions de films mais je n'ai pas réussi. J'ai essayé de récupérer des films plus récents... en vain. J'ai essayé de créer une vue matérialisé ... en vain.
J'aurais ensuite voulu tester un KNN Classifier mais là aussi pas eu le temps.

Bref mon système de recommandation est plutôt un échec.